# Predicting LPS vs Control Rats - Simple LSTM 
The goal of this is to demonstrate a pipeline for training LSTM. It is only a demonstration, there are a number of flaws with the design 

root


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path

tf.__version__

ModuleNotFoundError: No module named 'pandas'

## Load the Data

In [241]:
# Generated by DLC
control_file = Path(r"C:\Users\Owner\Documents\repos\COMP4905\local\sample\sample1 - Rat 4 ControlDLC_resnet50_MousePoseMay21shuffle1_13000.h5")
lps_file = Path(r"C:\Users\Owner\Documents\repos\COMP4905\local\sample\sample3 - Rat 2 LPSDLC_resnet50_MousePoseMay21shuffle1_13000.h5")

In [242]:
control_df = pd.read_hdf(control_file)
lps_df = pd.read_hdf(lps_file)

df.sample(3)

scorer    DLC_resnet50_MousePoseMay21shuffle1_13000                          \
bodyparts                                  left_ear                           
coords                                            x            y likelihood   
4                                       1375.858887  1149.991577   0.053112   
220                                     1392.278687  1283.067139   0.002469   
69                                      1818.094971   852.813538   0.042830   

scorer                                                                    \
bodyparts    right_ear                                 nose                
coords               x            y likelihood            x            y   
4          1282.761719  1114.820801   0.039103   920.545593  1157.993042   
220        1485.498169  1316.548462   0.006810  1504.992188  1150.591797   
69         1900.913452   876.447449   0.131736  1818.041138   847.618530   

scorer                             ...                                        \
bodyparts                  body_N  ... right_eye_N  right_eye_E                
coords    likelihood            x  ...  likelihood            x            y   
4           0.188549  1625.058472  ...    0.011127   923.095642  1161.811035   
220         0.012103  1624.804932  ...    0.044876  1459.582153   941.180908   
69          0.070660  1435.252441  ...    0.020964  1903.348389   885.694519   

scorer                                                                  \
bodyparts             right_eye_S                          right_eye_W   
coords    likelihood            x            y likelihood            x   
4           0.014954   913.486694  1162.501953   0.016795  1453.524170   
220         0.025480  1456.011230   938.725342   0.047699  1453.423462   
69          0.023286  1898.863281   887.140320   0.041443  1454.129395   

scorer                            
bodyparts                         
coords              y likelihood  
4          934.583557   0.019824  
220        936.166016   0.087085  
69         933.826050   0.042552  

[3 rows x 45 columns]

In [19]:
# Class labels from videos
control_df['label'] = 1
lps_df['label'] = 0

# Save index -> frame
control_df['frame'] = control_df.index
lps_df['frame'] = lps_df.index

array([1], dtype=int64)

# Create Train/Test
Since we're only using two videos (one for each class), we take the first part of each video as the train and the last as the test

In [38]:
import math

train = pd.concat([
    control_df[control_df.frame <= 200],
    lps_df[lps_df.frame <= 200]
], axis=0)

test = pd.concat([
    control_df[control_df.frame > 200],
    lps_df[lps_df.frame > 200]
], axis=0)

In [354]:
timesteps = 3
samples = 10000

start_frames = np.random.randint(train.frame.min(), train.frame.max() - steps + 1, size=samples)
train_y = np.random.randint(0, 2, size=samples)
train_X = np.zeros(shape=(samples, timesteps, input_shape))

for i in range(samples):
    sample = train[
        (train.label==train_y[i]) 
        & (train.frame > start_frames[i]) 
        & (train.frame <= start_frames[i] + timesteps)
    ]
    
    train_X[i] = sample.drop(columns=["frame", "label"]).values

In [355]:
# Find the smalled max frame value of the two classes
max_frame_cutoff = test.groupby('label').frame.max().min()

start_frames = np.random.randint(test.frame.min(), max_frame_cutoff - steps + 1, size=samples)
test_y = np.random.randint(0, 2, size=samples)
test_X = np.zeros(shape=(samples, timesteps, input_shape))

for i in range(samples):
    sample = test[
        (test.label==test_y[i]) 
        & (test.frame > start_frames[i]) 
        & (test.frame <= start_frames[i] + timesteps)
    ]
    
    test_X[i] = sample.drop(columns=["frame", "label"]).values

# Model

In [371]:
steps = 10
input_shape = 45

model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(100, return_sequences=True),
#     tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    tf.keras.optimizers.RMSprop(0.005, momentum=0.2),
#     loss=tf.keras.losses.BinaryCrossentropy()
    loss=tf.keras.losses.MeanSquaredError()
)

# Training

In [373]:
epochs = 1000
hist = []

for epoch in range(epochs):
    batch_indices = np.random.choice(np.arange(train_X.shape[0]), size=batch_size)
    batch_X = train_X[batch_indices]
    batch_y = train_y[batch_indices]
    
    if epoch%10==0:
        print(f"Batch: {epoch}")
        
    hist.append(model.fit(batch_X, batch_y, verbose=(epoch%10)==0 ))

Batch: 0
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2492
Batch: 10
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2536
Batch: 20
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2469
Batch: 30
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2468
Batch: 40
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2504
Batch: 50
Train on 32 samples
32/32 [==============================] - 0s 3ms/sample - loss: 0.2473
Batch: 60
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2450
Batch: 70
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2500
Batch: 80
Train on 32 samples
32/32 [==============================] - 0s 3ms/sample - loss: 0.2490
Batch: 90
Train on 32 samples
32/32 [==============================] - 0s 2ms/sample - loss: 0.2532
B

KeyboardInterrupt: 

# Test

In [374]:
pred = model.predict(test_X)[:,-1]
pred = pred.astype(float).reshape(-1)

accuracy = np.sum((pred > 0.5) == test_y) / pred.shape[0]
accuracy

0.498

In [351]:
p

(3000,)

In [325]:
test_y.shape

(1000,)

In [297]:
np.random.randint(0, 2, size=samples)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,